In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

# ----------------------------------------------------------------------
# 0. Chuẩn bị Dữ liệu: Đọc và Hợp nhất
# ----------------------------------------------------------------------

print("--- 0. Chuẩn bị Dữ liệu ---")

# Giả định các file đã được tải lên Colab và có thể đọc trực tiếp
try:
    df_train = pd.read_csv('Train.csv')
    df_test = pd.read_csv('Test.csv')
    df_meta = pd.read_csv('Meta.csv')
except FileNotFoundError as e:
    print(f"LỖI: Không tìm thấy file. Hãy đảm bảo bạn đã upload các file CSV lên Colab. Chi tiết: {e}")
    exit()

# Hợp nhất dữ liệu Train/Test với Meta data
# Merge dựa trên ClassId, vì ShapeId, ColorId, SignId là thuộc tính của ClassId
df_train = pd.merge(df_train, df_meta[['ClassId', 'ShapeId', 'ColorId', 'SignId']], on='ClassId', how='left')
df_test = pd.merge(df_test, df_meta[['ClassId', 'ShapeId', 'ColorId', 'SignId']], on='ClassId', how='left')

# Phân tách X và Y
X_train_raw = df_train.drop(['ClassId', 'Path'], axis=1)
y_train = df_train['ClassId']
X_test_raw = df_test.drop(['ClassId', 'Path'], axis=1)
y_test = df_test['ClassId']

# Dữ liệu thử nghiệm (validation) để huấn luyện Autoencoder
X_train_ae, X_val_ae, y_train_ae, y_val_ae = train_test_split(X_train_raw, y_train, test_size=0.2, random_state=42, stratify=y_train)


# ----------------------------------------------------------------------
# 1. Khám phá và Phân tích Dữ liệu (EDA)
# ----------------------------------------------------------------------

print("\n--- 1. Khám phá và Phân tích Dữ liệu (EDA) ---")
print(f"Kích thước tập huấn luyện: {df_train.shape}")
print(f"Kích thước tập kiểm tra: {df_test.shape}")
print("\nThông tin cấu trúc tập huấn luyện:")
X_train_raw.info()

# Phân tích Đặc trưng Mục tiêu (Target Variable)
print("\nPhân phối Đặc trưng Mục tiêu (ClassId):")
target_counts = y_train.value_counts().sort_index()
print(target_counts.head())
print(f"Số lượng lớp: {len(target_counts)}")
min_class = target_counts.min()
max_class = target_counts.max()
print(f"Mất cân bằng lớp: Lớp ít nhất ({min_class} mẫu) vs Lớp nhiều nhất ({max_class} mẫu).")

# Trực quan hóa (chạy trong Colab để hiển thị)
# plt.figure(figsize=(12, 5))
# sns.countplot(y=y_train, order=y_train.value_counts().index)
# plt.title('Phân phối số lượng mẫu theo ClassId (Kiểm tra mất cân bằng lớp)')
# plt.show()

# Phân tích Đặc trưng Số và Đặc trưng Phân loại
numerical_cols = ['Width', 'Height', 'Roi.X1', 'Roi.Y1', 'Roi.X2', 'Roi.Y2']
categorical_cols = ['ShapeId', 'ColorId', 'SignId']

# Thống kê mô tả đặc trưng số
print("\nThống kê mô tả Đặc trưng Số:")
print(X_train_raw[numerical_cols].describe())

# Kiểm tra dữ liệu thiếu (Missing Values)
print("\nKiểm tra Dữ liệu Thiếu:")
print(df_train.isnull().sum())
# Giả sử không có dữ liệu thiếu đáng kể sau khi merge (chỉ kiểm tra các cột mới)

# Phân tích mối quan hệ: Ma trận tương quan (Correlation Matrix)
# correlation_matrix = X_train_raw[numerical_cols].corr()
# plt.figure(figsize=(8, 6))
# sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
# plt.title('Ma trận tương quan giữa các Đặc trưng Số')
# plt.show()


# ----------------------------------------------------------------------
# 2. Tiền xử lý và Kỹ thuật Đặc trưng
# ----------------------------------------------------------------------

# Xác định lại các cột
NUMERICAL_COLS = ['Width', 'Height', 'Roi.X1', 'Roi.Y1', 'Roi.X2', 'Roi.Y2', 'Area', 'Roi_Width', 'Roi_Height', 'Roi_Area', 'Aspect_Ratio', 'Roi_Ratio']
META_COLS = ['ShapeId', 'ColorId', 'SignId']
TARGET_COL = 'ClassId'

# 2.1. Feature Engineering (Áp dụng cho cả Luồng A và Luồng B)

def create_new_features(df):
    """Tạo các đặc trưng mới từ bounding box và ROI."""
    df['Area'] = df['Width'] * df['Height']
    df['Roi_Width'] = df['Roi.X2'] - df['Roi.X1']
    df['Roi_Height'] = df['Roi.Y2'] - df['Roi.Y1']
    df['Roi_Area'] = df['Roi_Width'] * df['Roi_Height']
    # Tránh chia cho 0
    df['Aspect_Ratio'] = np.where(df['Height'] != 0, df['Width'] / df['Height'], 0)
    df['Roi_Ratio'] = np.where(df['Roi_Height'] != 0, df['Roi_Width'] / df['Roi_Height'], 0)
    return df

X_train_fe = create_new_features(X_train_raw.copy())
X_test_fe = create_new_features(X_test_raw.copy())


# --- Luồng A: Kỹ thuật Đặc trưng Truyền thống ---

# Lý giải lựa chọn:
# - Xử lý dữ liệu thiếu: Dùng Mode/Median cho các cột bị thiếu (nếu có, mặc dù đã merge và không có NaN)
# - Mã hóa: One-Hot Encoding cho các đặc trưng phân loại (ShapeId, ColorId, SignId) vì chúng là dữ liệu định danh (nominal). SignId được coi là mã danh mục.
# - Chuẩn hóa: StandardScaler được chọn vì nó chuẩn hóa dữ liệu xung quanh giá trị 0 với độ lệch chuẩn 1, phù hợp cho nhiều thuật toán, đặc biệt là các mô hình dựa trên gradient (như Logistic Regression).

# Khai báo Preprocessor cho Luồng A
preprocessor_A = ColumnTransformer(
    transformers=[
        # 1. Chuẩn hóa/Quy mô hóa Đặc trưng Số: StandardScaler
        ('num', StandardScaler(), NUMERICAL_COLS),
        # 2. Mã hóa Đặc trưng Phân loại: One-Hot Encoding
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), META_COLS)
    ],
    remainder='passthrough'
)

# Áp dụng Preprocessor Luồng A
X_train_A = preprocessor_A.fit_transform(X_train_fe)
X_test_A = preprocessor_A.transform(X_test_fe)

print("\nKích thước bộ đặc trưng Luồng A (Truyền thống):", X_train_A.shape)


# --- Luồng B: Học Đặc trưng bằng Autoencoder ---

# 2.2. Chuẩn bị dữ liệu cho Autoencoder
# Autoencoder cần input nằm trong khoảng [0, 1] để học tốt hơn, nên dùng MinMaxScaler cho toàn bộ dữ liệu.
# Sử dụng bộ tiền xử lý tương tự Luồng A, nhưng dùng MinMaxScaler.

preprocessor_B_AE = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), NUMERICAL_COLS),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), META_COLS)
    ],
    remainder='passthrough'
)

X_train_B_full = preprocessor_B_AE.fit_transform(X_train_fe)
X_test_B_full = preprocessor_B_AE.transform(X_test_fe)

input_dim = X_train_B_full.shape[1]
# Kích thước lớp cổ chai: Chọn giá trị nén (~1/3 đến 1/5 Input Dim), ở đây chọn 10.
# Lý giải: Kích thước 10 đảm bảo nén thông tin hiệu quả (giảm chiều dữ liệu từ ~40 về 10) nhưng đủ lớn để không mất mát quá nhiều chi tiết quan trọng.
BOTTLENECK_DIM = 10

# 2.3. Xây dựng và Huấn luyện Autoencoder
def build_autoencoder(input_dim, bottleneck_dim):
    # Encoder
    input_layer = Input(shape=(input_dim,))
    encoder = Dense(input_dim // 2, activation='relu')(input_layer)
    encoder = Dense(input_dim // 4, activation='relu')(encoder)
    bottleneck = Dense(bottleneck_dim, activation='relu', name='bottleneck')(encoder)

    # Decoder
    decoder = Dense(input_dim // 4, activation='relu')(bottleneck)
    decoder = Dense(input_dim // 2, activation='relu')(decoder)
    output_layer = Dense(input_dim, activation='sigmoid')(decoder) # Dùng sigmoid vì input đã được scale về [0, 1]

    # Model
    autoencoder = Model(inputs=input_layer, outputs=output_layer)
    return autoencoder, Model(inputs=input_layer, outputs=bottleneck) # Trả về Encoder riêng

# Quá trình huấn luyện:
# - Hàm mất mát (Loss): Mean Squared Error (MSE), đo lường sai số tái tạo.
# - Thuật toán tối ưu (Optimizer): Adam.
# - Số epochs: 50-100 (Chọn 50 để chạy nhanh)
# - Kích thước batch: 32

autoencoder, encoder = build_autoencoder(input_dim, BOTTLENECK_DIM)
autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

print("\nBắt đầu Huấn luyện Autoencoder (Luồng B)...")
history = autoencoder.fit(
    X_train_B_full, X_train_B_full, # X và Y là như nhau (dữ liệu đầu vào)
    epochs=50,
    batch_size=32,
    shuffle=True,
    validation_data=(X_test_B_full, X_test_B_full),
    verbose=0 # Tắt log để hiển thị gọn gàng
)
print("Huấn luyện Autoencoder hoàn tất. Loss cuối: {:.4f}".format(history.history['loss'][-1]))

# 2.4. Trích xuất Đặc trưng (Feature Extraction)
X_train_B = encoder.predict(X_train_B_full)
X_test_B = encoder.predict(X_test_B_full)

print("Kích thước bộ đặc trưng Luồng B (Autoencoder):", X_train_B.shape)


# ----------------------------------------------------------------------
# 3. Huấn luyện Mô hình & 4. Đánh giá và Phân tích Kết quả
# ----------------------------------------------------------------------

# Lựa chọn thuật toán:
# 1. Logistic Regression (mô hình tuyến tính)
# 2. Random Forest Classifier (mô hình cây/non-linear mạnh mẽ)

results = []

def evaluate_model(model_name, X_train, y_train, X_test, y_test, feature_flow):
    """Huấn luyện và đánh giá mô hình, trả về kết quả."""
    if model_name == 'Logistic Regression':
        model = LogisticRegression(max_iter=500, solver='sag', multi_class='multinomial', random_state=42)
    elif model_name == 'Random Forest':
        model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

    # Huấn luyện
    model.fit(X_train, y_train)

    # Dự đoán
    y_pred = model.predict(X_test)

    # Đánh giá
    accuracy = accuracy_score(y_test, y_pred)
    f1_macro = f1_score(y_test, y_pred, average='macro')

    print(f"\nKết quả {model_name} trên {feature_flow}:")
    print(classification_report(y_test, y_pred, zero_division=0))

    return {
        'Feature_Flow': feature_flow,
        'Model': model_name,
        'Accuracy': accuracy,
        'F1_Macro': f1_macro
    }

# 3.1. Huấn luyện trên Luồng A (Traditional Features)
print("\n" + "="*50)
print("HUẤN LUYỆN & ĐÁNH GIÁ (LUỒNG A: TRUYỀN THỐNG)")
results.append(evaluate_model('Logistic Regression', X_train_A, y_train, X_test_A, y_test, 'Luồng A'))
results.append(evaluate_model('Random Forest', X_train_A, y_train, X_test_A, y_test, 'Luồng A'))

# 3.2. Huấn luyện trên Luồng B (Autoencoder Features)
print("\n" + "="*50)
print("HUẤN LUYỆN & ĐÁNH GIÁ (LUỒNG B: AUTOENCODER)")
results.append(evaluate_model('Logistic Regression', X_train_B, y_train, X_test_B, y_test, 'Luồng B'))
results.append(evaluate_model('Random Forest', X_train_B, y_train, X_test_B, y_test, 'Luồng B'))

# 4. Lập bảng so sánh
results_df = pd.DataFrame(results)
print("\n" + "="*50)
print("BẢNG SO SÁNH HIỆU SUẤT TỔNG HỢP")
print(results_df.sort_values(by='Accuracy', ascending=False).to_markdown(index=False, floatfmt=".4f"))
print("="*50)


# ----------------------------------------------------------------------
# 5. So sánh và Kết luận
# ----------------------------------------------------------------------

best_model = results_df.loc[results_df['Accuracy'].idxmax()]
print("\n--- 5. So sánh và Kết luận ---")

print(f"Mô hình tốt nhất: {best_model['Model']} ({best_model['Feature_Flow']}) với Accuracy = {best_model['Accuracy']:.4f} và F1-Macro = {best_model['F1_Macro']:.4f}")

print("\n--- Phân tích Ưu và Nhược điểm ---")

print("Luồng A (Kỹ thuật Đặc trưng Truyền thống):")
print("- Ưu điểm: \n\t+ Tính diễn giải cao, dễ hiểu (ví dụ: biết Area, Aspect Ratio quan trọng).\n\t+ Quá trình tiền xử lý nhanh, không cần huấn luyện mạng nơ-ron.")
print("- Nhược điểm:\n\t+ Phụ thuộc vào kiến thức chuyên môn (domain knowledge) để tạo đặc trưng tốt.\n\t+ Độ dài vector đặc trưng có thể lớn (do One-Hot Encoding).")

print("\nLuồng B (Học Đặc trưng bằng Autoencoder):")
print(f"- Ưu điểm: \n\t+ **Giảm chiều dữ liệu hiệu quả** (từ {input_dim} về {BOTTLENECK_DIM} chiều), giảm nhiễu, chống overfitting.\n\t+ Tự động học các mối quan hệ phức tạp trong dữ liệu, không cần kiến thức chuyên môn sâu.")
print("- Nhược điểm:\n\t+ Tốn thời gian và tài nguyên để huấn luyện Autoencoder.\n\t+ Tính diễn giải (interpretability) thấp: khó hiểu ý nghĩa của từng chiều trong không gian đặc trưng mới.")

print("\n--- Kết luận về Hiệu quả ---")

# Dựa trên kết quả giả định (thường Random Forest trên Luồng A/B sẽ cao hơn Logistic Regression):
if best_model['Feature_Flow'] == 'Luồng A':
    print("Luồng A mang lại hiệu quả tốt hơn cho bài toán này.")
    print("Giả thuyết: Các đặc trưng thủ công (Area, Aspect Ratio) đã trích xuất đủ thông tin quan trọng một cách rõ ràng. Sự phức tạp của Autoencoder có thể đã nén quá mức hoặc không tìm thấy các mối quan hệ ẩn giá trị hơn các đặc trưng kỹ thuật thủ công đơn giản.")
else:
    print("Luồng B mang lại hiệu quả tốt hơn cho bài toán này.")
    print("Giả thuyết: Autoencoder đã thành công trong việc tạo ra một không gian đặc trưng **cô đọng** và **giàu thông tin** hơn. Bằng cách loại bỏ các thông tin dư thừa và chỉ giữ lại bản chất cốt lõi của dữ liệu (bottleneck layer), nó giúp mô hình học máy (đặc biệt là Random Forest) hoạt động hiệu quả hơn, ngay cả khi với số chiều dữ liệu thấp hơn nhiều so với Luồng A.")

--- 0. Chuẩn bị Dữ liệu ---

--- 1. Khám phá và Phân tích Dữ liệu (EDA) ---
Kích thước tập huấn luyện: (39209, 11)
Kích thước tập kiểm tra: (12630, 11)

Thông tin cấu trúc tập huấn luyện:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39209 entries, 0 to 39208
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Width    39209 non-null  int64 
 1   Height   39209 non-null  int64 
 2   Roi.X1   39209 non-null  int64 
 3   Roi.Y1   39209 non-null  int64 
 4   Roi.X2   39209 non-null  int64 
 5   Roi.Y2   39209 non-null  int64 
 6   ShapeId  39209 non-null  int64 
 7   ColorId  39209 non-null  int64 
 8   SignId   38759 non-null  object
dtypes: int64(8), object(1)
memory usage: 2.7+ MB

Phân phối Đặc trưng Mục tiêu (ClassId):
ClassId
0     210
1    2220
2    2250
3    1410
4    1980
Name: count, dtype: int64
Số lượng lớp: 43
Mất cân bằng lớp: Lớp ít nhất (210 mẫu) vs Lớp nhiều nhất (2250 mẫu).

Thống kê mô tả Đặc trưng Số:
     

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(



Kết quả Logistic Regression trên Luồng A:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        60
           1       0.22      0.45      0.29       720
           2       0.17      0.53      0.26       750
           3       0.16      0.03      0.06       450
           4       0.24      0.01      0.02       660
           5       0.26      0.11      0.15       630
           6       1.00      1.00      1.00       150
           7       0.00      0.00      0.00       450
           8       0.00      0.00      0.00       450
           9       1.00      1.00      1.00       480
          10       1.00      1.00      1.00       660
          11       1.00      1.00      1.00       420
          12       1.00      1.00      1.00       690
          13       1.00      1.00      1.00       720
          14       1.00      1.00      1.00       270
          15       1.00      1.00      1.00       210
          16       1.00      1.00     

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



Kết quả Logistic Regression trên Luồng B:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        60
           1       0.21      0.51      0.30       720
           2       0.17      0.42      0.24       750
           3       0.14      0.03      0.05       450
           4       0.15      0.11      0.13       660
           5       0.00      0.00      0.00       630
           6       1.00      1.00      1.00       150
           7       0.00      0.00      0.00       450
           8       0.00      0.00      0.00       450
           9       1.00      1.00      1.00       480
          10       1.00      1.00      1.00       660
          11       1.00      1.00      1.00       420
          12       1.00      1.00      1.00       690
          13       1.00      1.00      1.00       720
          14       1.00      1.00      1.00       270
          15       1.00      1.00      1.00       210
          16       1.00      1.00     